In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit

# Models
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

import math

import datetime
from datetime import datetime as dt
from dateutil.relativedelta import *
from datetime import timedelta

In [2]:
import dash
from dash import dcc, dash_table
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
import dash_bootstrap_components as dbc
import yfinance as yf
from datetime import datetime, timedelta
import talib as tb

In [3]:
df_coin = yf.download('BTC-USD',
                      end=datetime.today() - timedelta(days=1), 
                      progress=False,
    )
def df_converter(df): 
    df_sp500 = yf.download('^GSPC', 
                      start='2017-11-09', 
                      end='2022-05-08', 
                      progress=False)
    df_dollar = yf.download('DX=F', 
                      start='2017-11-09', 
                      end='2022-05-08', 
                      progress=False)
    # clearing dollar and sp500 df
    df_dollar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_dollar.rename(columns={"Close": "dollar_close"}, inplace=True)
    df_sp500.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_sp500.rename(columns={"Close": "sp500_close"}, inplace=True)
    # clearing general df
    #df_eth.drop('Unnamed: 0', axis=1, inplace=True)
    #df.drop('adj_close', axis=1, inplace=True)
    df.index = df.index.astype('datetime64[ns]')
    # MA df
    df_ma = df['Close'].to_frame()
    df_ma['SMA30'] = df_ma['Close'].rolling(15).mean()
    df_ma['CMA30'] = df_ma['Close'].expanding().mean()
    df_ma['EMA30'] = tb.EMA(df_ma['Close'], timeperiod=15)
    df_ma.dropna(inplace=True)
    # Stoch df
    slowk, slowd = tb.STOCH(df["High"], df["Low"], df["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df_stoch = pd.DataFrame(index=df.index,
                                data={"slowk": slowk,
                                    "slowd": slowd})
    df_stoch.dropna(inplace=True)
    # for later use in the concat
    stoch_c = ['slowk', 'slowd']
    # MACD df 
    macd, macdsignal, macdhist = tb.MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)
    df_macd = pd.DataFrame(index=df.index,
                            data={"macd": macd,
                                  "macdsignal": macdsignal,
                                  "macdhist": macdhist})
    df_macd.dropna(inplace=True)
    # for later use in the concat
    macd_c = ['macd', 'macdsignal', 'macdhist']
    # bb df
    upper, middle, lower = tb.BBANDS(df["Close"], timeperiod=15)
    df_bands = pd.DataFrame(index=df.index,
                                data={"bb_low": lower,
                                    "bb_ma": middle,
                                    "bb_high": upper})
    df_bands.dropna(inplace=True)
    # for later use in the concat
    bands_c = ['bb_low', 'bb_ma', 'bb_high']
    # rsi df
    rsi = tb.RSI(df['Close'], timeperiod=15)
    df_rsi = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "rsi": rsi})

    df_rsi.dropna(inplace=True)
    #stdev df
    stdev = tb.STDDEV(df['Close'], timeperiod=15, nbdev=1)
    df_stdev = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "stdev": stdev})
    df_stdev.dropna(inplace=True)
    # adx df
    adx = tb.ADX(df['High'], df['Low'], df['Close'], timeperiod=15)
    df_adx = pd.DataFrame(index=df.index,
                                data={"close": df['Close'],
                                    "adx": adx})

    df_adx.dropna(inplace=True)

    # concat 
    result =pd.concat([df, df_ma[['SMA30','CMA30','EMA30']], df_adx['adx'], df_bands[bands_c], df_macd[macd_c], df_rsi['rsi'], df_stdev['stdev'], df_stoch[stoch_c], df_dollar['dollar_close'], df_sp500['sp500_close']], axis=1)
    result.fillna(method='ffill', inplace=True)
    result.dropna(inplace=True)

    return result 

df_coin = df_converter(df_coin)
df_coin

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-09,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3226249984,6720.195964,1012.489040,6798.973215,37.428679,...,7929.899976,547.298468,521.809901,25.488567,65.836431,604.852006,33.184032,38.772059,94.351997,2584.620117
2017-11-10,7173.729980,7312.000000,6436.870117,6618.140137,6618.140137,5208249856,6767.749967,1017.359285,6776.369081,36.238180,...,7899.045668,482.149729,513.877867,-31.728137,55.203600,565.647850,30.022892,33.577955,94.278000,2582.300049
2017-11-11,6618.609863,6873.149902,6204.220215,6357.600098,6357.600098,4908680192,6806.196647,1021.994910,6724.022958,34.548539,...,7835.311479,404.828866,492.068066,-87.239201,50.841315,514.557416,13.668599,25.625174,94.278000,2582.300049
2017-11-12,6295.450195,6625.049805,5519.009766,5950.069824,5950.069824,8957349888,6819.328646,1026.269043,6627.278816,32.896648,...,7798.154128,307.126870,455.079827,-147.952957,44.895661,489.412741,14.127749,19.273080,94.278000,2582.300049
2017-11-13,5938.250000,6811.189941,5844.290039,6559.490234,6559.490234,6263249920,6846.371322,1031.063862,6618.805243,30.978407,...,7771.077715,275.694434,419.202748,-143.508315,53.591318,462.353197,27.607649,18.467999,94.388000,2584.840088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-24,29101.125000,29774.355469,28786.589844,29655.585938,29655.585938,26616506245,29782.070833,12250.832165,30662.861100,41.719564,...,31290.517495,-2174.369749,-2424.447237,250.077488,39.098232,754.223331,49.584235,47.287931,103.694000,4123.339844
2022-05-25,29653.134766,30157.785156,29384.949219,29562.361328,29562.361328,27525063551,29684.701172,12256.997240,30525.298628,41.149818,...,31041.086512,-2066.217894,-2352.801368,286.583474,38.850037,678.192670,36.400659,44.096835,103.694000,4123.339844
2022-05-26,29564.777344,29834.160156,28261.906250,29267.224609,29267.224609,36774325352,29706.759115,12263.052856,30368.039376,41.131924,...,31023.942631,-1981.480561,-2278.537207,297.056645,38.031135,658.591758,44.145672,43.376855,103.694000,4123.339844


In [4]:
df_coin = df_coin.tail(100)
df_coin.reset_index(inplace=True)
df_coin['Date'] = pd.to_datetime(df_coin['Date'], format='%Y-%m-%d')

In [5]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df_coin['Date'], y=df_coin['Close'], 
                name='Actual Values', mode='lines',line=dict(color='black')))
#Models
model_rf = RandomForestRegressor(random_state=10)
model_gb = GradientBoostingRegressor(random_state = 10)
model_xgb = XGBRegressor(random_state = 10)
#Predictions

In [6]:
def add_days(df, forecast_length):
    end_point = len(df)
    df1 = pd.DataFrame(index=range(forecast_length), columns=range(2))
    df1.columns = ['Close', 'Date']
    df = df.append(df1)
    df = df.reset_index(drop=True)
    x = df.at[end_point - 1, 'Date']
    x = pd.to_datetime(x, format='%Y-%m-%d')
    for i in range(forecast_length):
        df.at[df.index[end_point + i], 'Date'] = x + timedelta(days=1+i)
        df.at[df.index[end_point + i], 'close'] = 0
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df = df.drop(['Date'], axis=1)

In [7]:
def forecasting(model,df1, forecast_length):
    df3 = df1[['Close', 'Date']]
    df3 = add_days(df3, forecast_length)
    finaldf = df1.drop('Date', axis=1)
    finaldf = finaldf.reset_index(drop=True)
    end_point = len(finaldf)
    x = end_point - forecast_length
    finaldf_train = finaldf.loc[:x - 1, :]
    finaldf_train_x = finaldf_train.loc[:, finaldf_train.columns != 'Close']
    finaldf_train_y = finaldf_train['Close']

    fit = model.fit(finaldf_train_x, finaldf_train_y)
    yhat = []
    end_point = len(finaldf)
    df3_end = len(df3)
    for i in range(forecast_length, 0, -1):
        y = end_point - i
        inputfile = finaldf.loc[y:end_point, :]
        inputfile_x = inputfile.loc[:, inputfile.columns != 'Close']
        pred_set = inputfile_x.head(1)
        pred = fit.predict(pred_set)
        df3.at[df3.index[df3_end - i], 'Close'] = pred[0]
        finaldf = df1.drop('Date', axis=1)
        finaldf = finaldf.reset_index(drop=True)
        yhat.append(pred)
    yhat = np.array(yhat)
    return yhat

In [154]:
def portofolio (df_transactions, signal, value, date, df_coin, coin, df_summary):
    value_at_day = df_coin[df_coin.index == date]['Close'][0]
    percentage = value/value_at_day
    list_to_add = [coin, date, percentage, value, signal]
    df_length = len(df_transactions)
    df_transactions.loc[df_length] = list_to_add

    
    for coins in df_transactions['Coin'].unique():
        if coins == coin:
            if ~(df_summary['Coin'].str.contains(coins).any()):
                    for idx,percentage in enumerate(df_transactions['Percentage']):
                        df_summary.drop(df_summary.index[df_summary['Coin'] == coins], inplace=True)
                        df_length = len(df_summary)
                        actual_value = percentage * df_coin.tail(1)['Close'][0]
                        df_summary.loc[df_length] = [coins, percentage,actual_value,value]
            else:
                    value_to_add = 0
                    spent_to_add = 0
                    for idx,percentage in enumerate(df_transactions[df_transactions['Coin'] == coins]['Percentage']):
                        value_to_add += percentage
                        spent_to_add += df_transactions['Value'][idx]
                        print(spent_to_add)
                        actual_value = value_to_add * df_coin.tail(1)['Close'][0]
                        df_summary.drop(df_summary.index[df_summary['Coin'] == coins], inplace=True)
                        df_length = len(df_summary)
                        df_summary.loc[df_length] = [coins, value_to_add,actual_value,spent_to_add]

    return df_transactions, df_summary

In [155]:
def atualization(df_summary):
    value_list=[]
    for idx,coins in enumerate(df_summary['Coin'].unique()):
        df_coin = yf.download(coins, progress=False)
        value_to_add = df_coin.tail(1)['Close'][0] * df_summary[df_summary['Coin'] == coins]['Percentage'][idx]
        value_list.append(round(value_to_add,2))
    
    df_summary.drop('Value', axis=1, inplace=True)
    df_summary['Value'] = value_list
    return df_summary

In [156]:
df_summary = pd.DataFrame(columns=['Coin', 'Percentage','Value','Spent'])
df_transactions = pd.DataFrame(columns=['Coin', 'Date', 'Percentage', 'Value', 'Signal'])

In [157]:
data = pd.to_datetime('2022-05-24', format='%Y-%m-%d')

In [158]:
df_btc = yf.download('BTC-USD', progress=False)

In [159]:
df_transactions, df_summary = portofolio(df_transactions,0,10,data,df_btc,'BTC-USD',df_summary)
df_transactions

10


,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0


In [160]:
df_summary

,Coin,Percentage,Value,Spent
0,BTC-USD,0.000337,9.892408,10


In [161]:
df_transactions, df_summary = portofolio(df_transactions,0,10,data,df_btc,'BTC-USD',df_summary)
df_transactions

10
10
0
10
20


,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0


In [162]:
df_summary

,Coin,Percentage,Value,Spent
0,BTC-USD,0.000674,19.784816,20


In [163]:
df_sol = yf.download('SOL-USD', progress=False)

In [164]:
df_transactions, df_summary = portofolio(df_transactions,0,10,data,df_sol,'SOL-USD',df_summary)
df_transactions

20
20
20


,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0
2,SOL-USD,2022-05-24,0.201396,10,0


In [165]:
df_summary

,Coin,Percentage,Value,Spent
0,BTC-USD,0.000674,19.784816,20
1,SOL-USD,0.201396,9.088509,10


In [166]:
df_transactions, df_summary = portofolio(df_transactions,0,10,data,df_sol,'SOL-USD',df_summary)
df_transactions

20
10
0
10
20


,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0
2,SOL-USD,2022-05-24,0.201396,10,0
3,SOL-USD,2022-05-24,0.201396,10,0


In [167]:
df_summary

,Coin,Percentage,Value,Spent
0,BTC-USD,0.000674,19.784816,20
1,SOL-USD,0.402793,18.177018,20


In [168]:
df_transactions, df_summary = portofolio(df_transactions,0,10,data,df_sol,'SOL-USD',df_summary)
df_transactions

20
10
0
10
20
30


,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0
2,SOL-USD,2022-05-24,0.201396,10,0
3,SOL-USD,2022-05-24,0.201396,10,0
4,SOL-USD,2022-05-24,0.201396,10,0


In [169]:
df_summary

,Coin,Percentage,Value,Spent
0,BTC-USD,0.000674,19.784816,20
1,SOL-USD,0.604189,27.265528,30


In [170]:
df_near = yf.download('NEAR-USD', progress=False)

In [171]:
df_transactions, df_summary = portofolio(df_transactions,0,100,data,df_near,'NEAR-USD',df_summary)
df_transactions

20
20
20
20
20
20


,Coin,Date,Percentage,Value,Signal
0,BTC-USD,2022-05-24,0.000337,10,0
1,BTC-USD,2022-05-24,0.000337,10,0
2,SOL-USD,2022-05-24,0.201396,10,0
3,SOL-USD,2022-05-24,0.201396,10,0
4,SOL-USD,2022-05-24,0.201396,10,0
5,NEAR-USD,2022-05-24,16.771247,100,0


In [201]:
import plotly.express as px

color_discrete_map = {'Spent': 'rgb(30,144,255)', 'Value': 'rgb(0,0,255)'}
fig = px.histogram(df_summary, x="Coin", y=['Spent',"Value"],
              barmode='group',color_discrete_map=color_discrete_map,
             height=400)

fig.update_layout(
        template="plotly_dark",
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        paper_bgcolor = 'rgba(0, 0, 0, 0)',
        font_color="white",
        font_size= 10,
        margin={'t': 0,'l':0,'b':10,'r':0})
fig.show()